# Final - 1 WebScraping with Beautiful Soup of DCI Mueseum

This file is extra code showing how I got to the final version of the code.

In [3]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

page = requests.get("http://www.dcxmuseum.org/repertoire_display.cfm?ReturnAll=Y&RepYear=2022&Decade=2020")
soup = BeautifulSoup(page.content, 'html.parser')

In [2]:
page.status_code
#200 = HTTP request recieved, understood, and processed = success!

200

In [179]:
print(soup.prettify()) #spoiler, much more readable than page.content!
#we see information we want such as date/score/etc. in the 'td' tag

<div>
 Return to
 <button class="btn btn-info" onclick="scoreDisplayAll(17)" type="button">
  All Years
 </button>
</div>
<table class="table table-bordered table-striped">
 <thead>
  <tr>
   <th colspan="4">
    View All Scores for
    <a href="javascript:{}" onclick="showChangeYear(2019, -2, 17)">
     2019
    </a>
   </th>
  </tr>
  <tr>
   <th>
    Date
   </th>
   <th>
    Show
   </th>
   <th>
    Position
   </th>
   <th>
    Score
   </th>
  </tr>
 </thead>
 <tbody>
  <tr valign="top">
   <td>
    June 21
   </td>
   <td>
    <b>
     <span style="color:red">
      MidCal Champions Showcase
     </span>
    </b>
    <a href="javascript:{}" onclick="scoreDisplayYear(17,2019,2019002)">
     Clovis CA United States
    </a>
    <font size="-2">
     <b>
      DCI
     </b>
    </font>
   </td>
   <td align="right">
    2
   </td>
   <td align="right">
    71.700
   </td>
  </tr>
  <tr valign="top">
   <td>
    June 22
   </td>
   <td>
    <b>
     <span style="color:red">
      D

# Finding/Cleaning the text in a html tag

In [12]:
#get a [LIST] of 'td' tags
#if we count, we see the 2nd item is Blue Devils
soup.find_all("td")

[<td colspan="4" style="background-color:black; color:white">Junior</td>,
 <td width="25%">
 <a href="index.cfm?view=corpslist&amp;corpsid=17&amp;corpsyear=2022">Blue Devils</a></td>,
 <td align="right">1</td>,
 <td align="right">  98.750</td>,
 <td>
 <font color="RED">Tempus Blue</font> -
 	<a href="index.cfm?view=search&amp;song=Lapus%20Luzi">Lapus Lazuli</a>
 		* <a href="index.cfm?view=search&amp;song=Divine%20Blue">Divine Blue</a>
 		* <a href="index.cfm?view=search&amp;song=Carnival%20of%20Venice%20">Carnival of Venice </a>
 		* <a href="index.cfm?view=search&amp;song=Moon%20River">Moon River</a>
 		* <a href="index.cfm?view=search&amp;song=Hands%20and%20Feet">Hands and Feet</a>
 		* <a href="index.cfm?view=search&amp;song=Silk%20Sonic%20Intro">Silk Sonic Intro</a>
 		* <a href="index.cfm?view=search&amp;song=Tank%21">Tank!</a>
 		* <a href="index.cfm?view=search&amp;song=Home%20of%20the%20Brave">Home of the Brave</a>
 		
 	 
 	</td>,
 <td width="25%">
 <a href="index.cfm?view=co

In [11]:
#look at the 2nd item it shows blue devils (remember python starts at 0 for the 1st item)
soup.find_all("td")[1]

<td width="25%">
<a href="index.cfm?view=corpslist&amp;corpsid=17&amp;corpsyear=2022">Blue Devils</a></td>

In [15]:
#use .text to extract the text, but there is a whitespace character - \n
soup.find_all("td")[1].text

'\nBlue Devils'

In [16]:
#.strip will remove things at the beginning or end
soup.find_all("td")[1].text.strip('\n')

'Blue Devils'

In [19]:
#this example shows show name and also the list of songs
soup.find_all('td')[4].text

'\nTempus Blue -\r\n\tLapus Lazuli\r\n\t\t* Divine Blue\r\n\t\t* Carnival of Venice \r\n\t\t* Moon River\r\n\t\t* Hands and Feet\r\n\t\t* Silk Sonic Intro\r\n\t\t* Tank!\r\n\t\t* Home of the Brave\r\n\t\t\r\n\t\xa0\r\n\t'

#### Remove Whitespace Characters

In [20]:
#to remove most of those whitespace characters, use regular expression and do substitution
import re
re.sub('[\n\t*-]', '', soup.find_all('td')[4].text)
#4 things are being replaced with nothing: \n \t * -
#sub syntax: 1. what to look for, 2. what to replace it with, 3. the string being operated on.

'Tempus Blue \rLapus Lazuli\r Divine Blue\r Carnival of Venice \r Moon River\r Hands and Feet\r Silk Sonic Intro\r Tank!\r Home of the Brave\r\r\xa0\r'

In [21]:
#now we are left with \r which we can split on to get a list of individual songs
re.sub('[\n\t*-]', '', soup.find_all('td')[4].text).split('\r')

['Tempus Blue ',
 'Lapus Lazuli',
 ' Divine Blue',
 ' Carnival of Venice ',
 ' Moon River',
 ' Hands and Feet',
 ' Silk Sonic Intro',
 ' Tank!',
 ' Home of the Brave',
 '',
 '\xa0',
 '']

In [24]:
#the first item in the list is the show name, the rest are songs
#lets break that up and also remove the junk at the end
print(re.sub('[\n\t*-]', '', soup.find_all('td')[4].text).split('\r')[0:1]) #show name
print(re.sub('[\n\t*-]', '', soup.find_all('td')[4].text).split('\r')[1:-3]) #songs without junk at the end

['Tempus Blue ']
['Lapus Lazuli', ' Divine Blue', ' Carnival of Venice ', ' Moon River', ' Hands and Feet', ' Silk Sonic Intro', ' Tank!', ' Home of the Brave']


#### The page repeates every 4 items:

-Corps

-Rank

-Score

-Show-Songs

# Dictionary Corps:ID

In [4]:
#make a dict of corps names to their id
page = requests.get("http://www.dcxmuseum.org/repertoire_display.cfm?ReturnAll=Y&RepYear=2019&Decade=2010")
soup = BeautifulSoup(page.content, 'html.parser')

tags = soup.find_all('a')
corpsids = {}
for tag in tags:
    attr = tag.attrs['href']
    if 'corpsid' in attr:
        attr = re.sub('.*corpsid=','',attr)
        attr = re.sub('&.*','',attr)
        corpsids[tag.string]=attr

corpsids['Blue Devils'] #case sensitive search for id of ['corps_name']
corpsids #dict of corps_name:corps_id

{'Blue Devils': '17',
 'Bluecoats': '32',
 'Santa Clara Vanguard': '3',
 'Carolina Crown': '43',
 'The Cavaliers': '9',
 'Boston Crusaders': '37',
 'Blue Knights': '34',
 'Blue Stars': '2',
 'The Cadets': '21',
 'Mandarins': '48',
 'Crossmen': '25',
 'Phantom Regiment': '19',
 'Spirit of Atlanta': '26',
 'Pacific Crest': '108',
 'The Academy': '1455',
 'Colts': '36',
 'Madison Scouts': '13',
 'Troopers': '6',
 'Spartans': '66',
 'Genesis': '1645',
 'Legends': '1612',
 'Gold': '1572',
 'Music City': '1624',
 'Jersey Surf': '93',
 'Seattle Cascades': '125',
 'Louisiana Stars': '1765',
 'Southwind': '40',
 'River City Rhythm': '1781',
 'Guardians': '1749',
 '7th Regiment': '1403',
 'Golden Empire': '1764',
 'Shadow': '1821',
 'Colt Cadets': '80',
 'Les Stentors': '177',
 'Raiders': '116',
 'Heat Wave ': '1777',
 'Blue Devils B': '71',
 'Blue Devils C': '70',
 'Columbians': '1748',
 'Encorps': '2663',
 'Impulse': '497',
 'Incognito': '1617',
 'Oregon Crusaders': '670',
 'The Battalion': '1

In [29]:
#make a list of corps_names and a list of corps_ids
clist = list(corpsids.keys())
cidlist = list(corpsids.values())
print('Corps IDs: \n',cidlist)
print('\nCorps List: \n',clist)
#print('\nDictionary of Corps and Ids:\n',corpsids)

Corps IDs: 
 ['17', '32', '3', '43', '9', '37', '34', '2', '21', '48', '25', '19', '26', '108', '1455', '36', '13', '6', '66', '1645', '1612', '1572', '1624', '93', '125', '1765', '40', '1781', '1749', '1403', '1764', '1821', '80', '177', '116', '1777', '71', '70', '1748', '2663', '497', '1617', '670', '1816', '398', '1653', '1766', '1851', '1971', '1853', '2299', '1856', '2527', '2104', '1857', '2638', '1858', '2117', '1868', '1775', '1867', '2094', '3001', '1956', '1988', '1954', '555', '1272', '2578', '98']

Corps List: 
 ['Blue Devils', 'Bluecoats', 'Santa Clara Vanguard', 'Carolina Crown', 'The Cavaliers', 'Boston Crusaders', 'Blue Knights', 'Blue Stars', 'The Cadets', 'Mandarins', 'Crossmen', 'Phantom Regiment', 'Spirit of Atlanta', 'Pacific Crest', 'The Academy', 'Colts', 'Madison Scouts', 'Troopers', 'Spartans', 'Genesis', 'Legends', 'Gold', 'Music City', 'Jersey Surf', 'Seattle Cascades', 'Louisiana Stars', 'Southwind', 'River City Rhythm', 'Guardians', '7th Regiment', 'Golden

In [6]:
cidlist= list(corpsids.values())
print('Corps IDs: \n',cidlist)

Corps IDs: 
 ['17', '32', '3', '43', '9', '37', '34', '2', '21', '48', '25', '19', '26', '108', '1455', '36', '13', '6', '66', '1645', '1612', '1572', '1624', '93', '125', '1765', '40', '1781', '1749', '1403', '1764', '1821', '80', '177', '116', '1777', '71', '70', '1748', '2663', '497', '1617', '670', '1816', '398', '1653', '1766', '1851', '1971', '1853', '2299', '1856', '2527', '2104', '1857', '2638', '1858', '2117', '1868', '1775', '1867', '2094', '3001', '1956', '1988', '1954', '555', '1272', '2578', '98']


# List of Years to Scrape (as string)

In [7]:
#make a list of years 2000-2019
ylist= []
for year in range (2000,2020):
    ylist.append(year)
    
ylist = list(map(str,ylist)) #format the list of int as a list of strings instead
print(ylist)

['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']


# Final Scores - Fixed Year, All Corps

In [40]:
#Keep
#finals scores for all corps in a given year
#cyclimatic complexity - number of branches in code

year = '2019'
page = requests.get("http://www.dcxmuseum.org/repertoire_display.cfm?ReturnAll=Y&RepYear="+year+"&Decade=2010")
soup = BeautifulSoup(page.content, 'html.parser')

#Create df
l_final_yr=[] #list of final scores for fixed year
stop=len(soup.find_all('td'))-1 #use len() to find the stopping point, idk why -1 it errors out otherwise
for x in range(1,stop,4):
    if soup.find_all("td")[x].text.strip('\n') == 'Senior':
        break #break loop at 'Senior' class division
    l_final_yr.append((
    soup.find_all("td")[x].text.strip('\n'),#corps
    year,
    soup.find_all("td")[x+1].text.strip('\n'),#rank
    soup.find_all("td")[x+2].text.strip('\n'),#score
    re.sub('[\n\t*-]', '', soup.find_all('td')[x+3].text).split('\r')[0:1],#show
    re.sub('[\n\t*]', '', soup.find_all('td')[x+3].text).split('\r')[1:-3]#songs
    ))

df_final_yr = pd.DataFrame(l_final_yr, columns=('Corps','Year','Rank','Final Score','Show','Songs'))
df_final_yr


,Corps,Year,Rank,Final Score,Show,Songs
0,Blue Devils,2019,1,98.325,[Ghostlight ],"[Don't Think, Cycle Song, Rhythm Song, A Tr..."
1,Bluecoats,2019,2,98.238,[The Bluecoats ],"[Revolution Number 9, Strawberry Fields Forev..."
2,Santa Clara Vanguard,2019,3,96.600,[Vox Eversio ],"[Fraternity, Audivi Media Nocte, Nothing Els..."
3,Carolina Crown,2019,4,96.563,[Beneath the Surface ],"[Temen Oblak (Dark Clouds), Frolic (From Rabb..."
4,The Cavaliers,2019,5,95.400,[The Wrong Side of the Tracks ],"[Gorgon, Bum's Rush, Rock Island, 1931, Par..."
5,Boston Crusaders,2019,6,94.488,[Goliath ],"[Enough of Our Machines, Everybody Wants to R..."
6,Blue Knights,2019,7,92.050,[...I Remember Everything ],"[Sudden Throw , Liquid, Hurt, Lonely World]"
7,Blue Stars,2019,8,91.225,[Call of the Wild ],"[Winter, Symphony No. 1 , Dance Panels , S..."
8,The Cadets,2019,9,89.838,[Behold! ],"[Fanfare for the Women, Just, Blueprint, Da..."
9,Mandarins,2019,10,89.300,[Subterra ],"[Ritual and Discovery, On the Shoulders of Gi..."


# Final Form (Final Scores)
# Final Scores - Range of Years, All Corps

In [39]:
#finals scores for range of years 2000-2019
#change decade
l_final=[] #list of final scores
for year in ylist:
    decade = str(int(int(year)/10)*10) #converting from string to int, then casting a decimal back to int again, floors it.
    page = requests.get("http://www.dcxmuseum.org/repertoire_display.cfm?ReturnAll=Y&RepYear="+year+"&Decade="+decade)
    soup = BeautifulSoup(page.content, 'html.parser')
    stop=len(soup.find_all('td'))-1
    
    for x in range(1,stop,4):
        if soup.find_all("td")[x].text.strip('\n') == 'Senior':
            break
        l_final.append((
        soup.find_all("td")[x].text.strip('\n'),#corps
        year,
        soup.find_all("td")[x+1].text.strip('\n'),#rank
        soup.find_all("td")[x+2].text.strip('\n'),#score
        re.sub('[\n\t*-]', '', soup.find_all('td')[x+3].text).split('\r')[0:1],#show
        re.sub('[\n\t*]', '', soup.find_all('td')[x+3].text).split('\r')[1:-3]#songs
        ))
df_final = pd.DataFrame(l_final, columns=('Corps','Year','Rank','Final Score','Show','Songs'))
df_final

,Corps,Year,Rank,Final Score,Show,Songs
0,The Cadets,2000,1,97.650,[We are the Future ],[Tapestry Of Nations (Prologue from Mov. III o...
1,The Cavaliers,2000,1,97.650,[Niagara Falls ],"[Niagara Falls, Original Composition]"
2,Blue Devils,2000,3,96.900,[Methods of Madness ],[A Night Piece For Orchestra..Prelude & Blues ...
3,Santa Clara Vanguard,2000,4,94.700,[The Age of Reverence ],"[Prayers of Kierkegaard, Prayer No. 4, String..."
4,Boston Crusaders,2000,5,92.350,[RED ],"[Bolero, Intensity, Time To Say Goodbye, Da..."
...,...,...,...,...,...,...
1071,Oregon Crusaders,2019,,,[Corps inactive],[]
1072,The Battalion,2019,,,[Humanity ],"[Shenandoah, Many Languages, One World, Sunk..."
1073,Vanguard Cadets,2019,,,[What About Us ],"[What About Us, Primacy of Number (from Naqoy..."
1074,Vessel,2019,,,"[Here, There, Everywhere ]","[Theme from Super 8, The Phoenix, Rewrite th..."


# Season Scores - Fixed Year, Fixed Corps

In [22]:
#season scores given year given corps
year = '2000'
corps = 'Carolina Crown' #case sensitive
page = requests.get("http://www.dcxmuseum.org/Corpslist_ScoreYear.cfm?ReturnAll=Y&CorpsID="+corpsids[corps]+"&CorpsYear="+year+"&ShowID=0")
soup = BeautifulSoup(page.content, 'html.parser')

l_season_cpyr=[] #list of season scores for fixed corps/year
stop=len(soup.find_all('td')) #use len to find the stopping point of the loop

for x in range(0,stop,4):
    l_season_cpyr.append((
    corps,
    soup.find_all("td")[x].text, #show date
    year,
    re.sub('\n',', ',soup.find_all("td")[x+1].text), #show details
    soup.find_all("td")[x+2].text, #show position
    re.sub('\r\n\t\t','',soup.find_all("td")[x+3].text).strip() #show score
    ))
    
df_season_cpyr = pd.DataFrame(l_season_cpyr, columns=('Corps','Date','Year','Show','Position ','Show Score'))
df_season_cpyr

,Corps,Date,Year,Show,Position,Show Score
0,Carolina Crown,June 23,2000,", Atlanta GA , DCI Atlantic",3,62.450
1,Carolina Crown,June 25,2000,", Orlando FL , DCI",2,66.000
2,Carolina Crown,June 26,2000,", Tampa FL , DCI Atlantic",3,66.700
3,Carolina Crown,June 27,2000,", Brunswick GA , DCI Atlantic",3,67.150
4,Carolina Crown,June 28,2000,", Duncan SC , DCI Atlantic",2,68.450
5,Carolina Crown,June 29,2000,", Lynchburg VA , DCI Atlantic",3,68.900
6,Carolina Crown,June 30,2000,", Elkton MD , DCI Atlantic",3,67.700
7,Carolina Crown,July 1,2000,", Clifton NJ , DCI Atlantic",3,68.850
8,Carolina Crown,July 2,2000,", Beverly MA , DCI Atlantic",2,71.650
9,Carolina Crown,July 3,2000,", Bristol RI , DCI Atlantic",3,72.900


# Season Scores - Range of Years, Fixed Corps

In [43]:
#keep
#Carolina Crown show scores over 20 years (from ylist)
corps = 'Carolina Crown'
l_season_cp=[] #list of season scores for fixed corps

for year in ylist:
    page = requests.get("http://www.dcxmuseum.org/Corpslist_ScoreYear.cfm?ReturnAll=Y&CorpsID="+corpsids[corps]+"&CorpsYear="+year+"&ShowID=0")
    soup = BeautifulSoup(page.content, 'html.parser')
    stop=len(soup.find_all('td')) #use len to find the stopping point of the loop

    for x in range(0,stop,4):
        l_season_cp.append((
        corps,
        soup.find_all("td")[x].text+', '+year, #show date
        year,
        re.sub('\n',', ',soup.find_all("td")[x+1].text), #show details
        soup.find_all("td")[x+2].text, #show position
        re.sub('\r\n\t\t','',soup.find_all("td")[x+3].text).strip() #show score
        ))
        
df_season_cp = pd.DataFrame(l_season_cp, columns=('Corps','Date','Year','Show','Position ','Show Score'))
df_season_cp

,Corps,Date,Year,Show,Position,Show Score
0,Carolina Crown,"June 23, 2000",2000,", Atlanta GA , DCI Atlantic",3,62.450
1,Carolina Crown,"June 25, 2000",2000,", Orlando FL , DCI",2,66.000
2,Carolina Crown,"June 26, 2000",2000,", Tampa FL , DCI Atlantic",3,66.700
3,Carolina Crown,"June 27, 2000",2000,", Brunswick GA , DCI Atlantic",3,67.150
4,Carolina Crown,"June 28, 2000",2000,", Duncan SC , DCI Atlantic",2,68.450
...,...,...,...,...,...,...
633,Carolina Crown,"August 4, 2019",2019,"Drums Along the Waterfront, Buffalo NY United ...",2,94.775
634,Carolina Crown,"August 5, 2019",2019,"Tour of Champions ? Akron, Akron OH United Sta...",3,94.850
635,Carolina Crown,"August 8, 2019",2019,"DCI World Championship Prelims, Indianapolis I...",4,95.938
636,Carolina Crown,"August 9, 2019",2019,"DCI World Championship Semifinals, Indianapoli...",4,96.025


# Final Form (Season Scores):
# Season Scores - Range of Years, All Corps

In [44]:
#Runs for a while
#All junior corps show scores over 20 years
l_season=[]
for corps in clist:
    for year in ylist:
        page = requests.get("http://www.dcxmuseum.org/Corpslist_ScoreYear.cfm?ReturnAll=Y&CorpsID="+corpsids[corps]+"&CorpsYear="+year+"&ShowID=0")
        soup = BeautifulSoup(page.content, 'html.parser')
        stop=len(soup.find_all('td')) #use len to find the stopping point of the loop

        for x in range(0,stop,4):
            l_season.append((
            corps,
            soup.find_all("td")[x].text+', '+year, #show date
            year,
            re.sub('\n',', ',soup.find_all("td")[x+1].text), #show details
            soup.find_all("td")[x+2].text, #show position
            re.sub('\r\n\t\t','',soup.find_all("td")[x+3].text).strip() #show score
            ))
        
df_season = pd.DataFrame(l_season, columns=('Corps','Date','Year','Show','Position ','Show Score'))
df_season

,Corps,Date,Year,Show,Position,Show Score
0,Blue Devils,"June 17, 2000",2000,", Concord CA , DCI-Pacific",1,73.550
1,Blue Devils,"June 23, 2000",2000,", Glendora CA , DCI Pacific",1,79.000
2,Blue Devils,"June 24, 2000",2000,", Riverside CA , DCI Pacific",1,80.050
3,Blue Devils,"June 25, 2000",2000,", Costa Mesa CA , DCI Pacific",1,80.800
4,Blue Devils,"July 21, 2000",2000,", Des Plaines IL , DCI",1,88.600
...,...,...,...,...,...,...
17476,Freelancers Alumni Minicorps,"June 24, 2018",2018,"Moonlight Classic, Sacramento CA United States...",0,
17477,Freelancers Alumni Minicorps,"August 31, 2018",2018,"I&E and Mini Corps, Williamsport PA United Sta...",2,94.667
17478,Freelancers Alumni Minicorps,"September 2, 2018",2018,"Alumni Spectacular, Williamsport PA United Sta...",0,
17479,Freelancers Alumni Minicorps,"August 30, 2019",2019,"I&E and Mini Corps, Williamsport PA United Sta...",1,97.170


# Corps City and State

In [ ]:
#scrape corps home town
page = requests.get("http://www.dcxmuseum.org/index.cfm?roomid=105&view=corps&option=junior")
soup = BeautifulSoup(page.content, 'html.parser')
stop=len(soup.find_all('td'))-1
chome = []
for x in range(5,stop,5):
    chome.append((
    soup.find_all("td")[x].text.strip('\n'),#corps
    re.sub('[\n\t\r]', '', soup.find_all('td')[x+2].text),#town
    re.sub('[\n\t\r ]', '', soup.find_all('td')[x+3].text)#state
    ))
df_chome = pd.DataFrame(chome, columns=('Corps','City','State'))
df_chome=df_chome[df_chome['Corps'].isin(clist)]
df_chome

In [ ]:
#added a lat/long column (initially set to 0)
df_chome.insert(3,'Latitude',0)
df_chome.insert(4,'Longitude',0)

In [ ]:
#update lat/long using geopy library
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="x2x")

for index,row in df_chome.iterrows():
    location = geolocator.geocode(row['City'] + row['State'])
    #error handling for unknown geolocation lookups
    if location is None:
        df_chome.loc[index,['Latitude','Longitude']]=[0,0]
    else:
        df_chome.loc[index,['Latitude','Longitude']]=[location.latitude, location.longitude]
df_chome  

# Common HTML tags

* __\<head> \</head>__ - defines the head of a document
* __\<body> \</body>__ - definse the body of a document
* __\<p> \</p>__ - defines a paragraph
* __\<a> \</a>__ - used for links with "href=_url_"

# Common HTML tags

* __\<div> \</div>__ - delineates a page division
* __\<table> \</table>__ - creates a table
* __\<form> \</form>__ - creates a form to take input
* __\<b> \</b>__ - bold text
* __\<i> \</i>__ - italicize text

# HTML class & id

HTML have properties __class__ and __id__ that give names to html elements.

* __class__ can be used by multiple elements, and single elements can have multiple classes.
* __id__ can only be used by a single element per page, and an element can only have a single id.

Think of __class__ and __id__ as metadata for html elements.

In [ ]:
#get table info
cursor = connection.execute('pragma table_info(''final_scores'')')
for row in cursor:
    print(row)

In [ ]:
#example query
cursor = connection.execute('select Corps, Rank,Year, Final from final_scores where Corps = "Carolina Crown"')
for row in cursor:
    print(row)

In [ ]:
#look at tables in given database
database = 'dci.db'
connection = sqlite3.connect(database)
cursor = connection.execute('SELECT name FROM sqlite_schema WHERE type=\'table\' ORDER BY name')
for row in cursor:
    print(row)

# Helpful Links

Hacking the API - Greg Greda

http://www.gregreda.com/2015/02/15/web-scraping-finding-the-api/


Regular Expression Python Docs

https://docs.python.org/3/library/re.html

List Index out of range - knowing when to stop the loop based on size of page being scraped.

https://www.freecodecamp.org/news/list-index-out-of-range-python-error-message-solved/

Changing the values in a column of a dataframe (from date = MMDD to MMDDYYYY)

https://www.statology.org/pandas-select-column-by-index/


Add a column to a dataframe

https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/


convert year_list (list of ints) to (list of str)

https://www.geeksforgeeks.org/python-program-to-convert-list-of-integer-to-list-of-string/


update update value per row (section 4)

https://www.digitalocean.com/community/tutorials/update-rows-and-columns-python-pandas

geomap

https://pypi.org/project/geopy/

# Helpful Plotting Links


vid of sunburst

https://www.youtube.com/watch?v=RdaV9dvC6sc

sunburst syntax

https://plotly.com/python-api-reference/generated/plotly.express.sunburst.html

sunburst color options
 
https://plotly.com/python/discrete-color/

Sunburst

https://plotly.com/python/sunburst-charts/

matplotlib marker options

https://matplotlib.org/stable/api/markers_api.html

matplotlib line color options

https://matplotlib.org/stable/gallery/color/named_colors.html